In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
import numpy as np
import random
import time
import os
import sys
import mat73
from torchsummary import summary

from src.models.ref_dual_path_autoencoder import RefDualPathAutoencoder
from src.training.train_ref_dp_model import train_refdual_model

In [3]:
# Set absolute path for the project root directory
PROJECT_ROOT = r'c:\Users\WangCo\Documents\ProgramData\Python\radar_interference_suppression'

# Ensure the project root directory is in the Python path
if PROJECT_ROOT not in sys.path:
    sys.path.append(PROJECT_ROOT)

# Verify the path settings
print("Current working directory:", os.getcwd())
print("Project root directory:", PROJECT_ROOT)
print("Python path:", sys.path)

# Check if directories exist
expected_dirs = ['src', 'datasets', 'notebooks', 'outputs', 'configs']
for dir_name in expected_dirs:
    dir_path = os.path.join(PROJECT_ROOT, dir_name)
    print(f"'{dir_name}' directory exists:", os.path.exists(dir_path))

# Change working directory to project root
os.chdir(PROJECT_ROOT)
print("\nNew working directory:", os.getcwd())

Current working directory: c:\Users\WangCo\Documents\ProgramData\Python\radar_interference_suppression\notebooks
Project root directory: c:\Users\WangCo\Documents\ProgramData\Python\radar_interference_suppression
Python path: ['c:\\Users\\WangCo\\AppData\\Local\\Programs\\Python\\Python311\\python311.zip', 'c:\\Users\\WangCo\\AppData\\Local\\Programs\\Python\\Python311\\DLLs', 'c:\\Users\\WangCo\\AppData\\Local\\Programs\\Python\\Python311\\Lib', 'c:\\Users\\WangCo\\AppData\\Local\\Programs\\Python\\Python311', '', 'C:\\Users\\WangCo\\AppData\\Roaming\\Python\\Python311\\site-packages', 'C:\\Users\\WangCo\\AppData\\Roaming\\Python\\Python311\\site-packages\\win32', 'C:\\Users\\WangCo\\AppData\\Roaming\\Python\\Python311\\site-packages\\win32\\lib', 'C:\\Users\\WangCo\\AppData\\Roaming\\Python\\Python311\\site-packages\\Pythonwin', 'c:\\Users\\WangCo\\AppData\\Local\\Programs\\Python\\Python311\\Lib\\site-packages', 'c:\\Users\\WangCo\\Documents\\ProgramData\\Python\\radar_interference_

In [25]:
import importlib
# import src
from src.utils import analysis
from src.utils import training_validation
from src.utils import evaluation
from src.data import data_utils
from src.utils import visualization
from src.data.dataset import RadarDataset
from src.models import baseline_tec
from src.models import ref_dual_path_autoencoder
from src.training import train_ref_model
from src.utils import evaluation_ref_dp


In [30]:
# # # Reload the modules to get the latest changes
# sys.path.append(os.path.abspath('..'))

# # # # Reload the modules to get the latest changes
# importlib.reload(src)
importlib.reload(data_utils)
importlib.reload(visualization)
importlib.reload(analysis)
importlib.reload(training_validation)
importlib.reload(evaluation)
importlib.reload(evaluation_ref_dp)
importlib.reload(ref_dual_path_autoencoder)
importlib.reload(train_ref_model)

print("Modules reloaded successfully!")
# # # Reload the modules to get the latest changes

Modules reloaded successfully!


In [10]:
scenario_files = [
    os.path.join(PROJECT_ROOT, 'datasets', 'raw', 'test_dataset_2t1_60f.mat')  
]

# Load and analyze all scenarios
print("Loading scenario data...")
radar_cube, clean_data, scenario_info = data_utils.load_multiple_scenarios(scenario_files)

# DEBUG INFO
print(f"Input shapes - radar_cube: {radar_cube.shape}, clean_data: {clean_data.shape}")
print(f"Data types - radar_cube: {radar_cube.dtype}, clean_data: {clean_data.dtype}")

# Check data values
print("\nData statistics:")
print(f"Radar cube - Mean amplitude: {np.mean(np.abs(radar_cube)):.3f}")
print(f"Radar cube - Max amplitude: {np.max(np.abs(radar_cube)):.3f}")
print(f"Clean data - Mean amplitude: {np.mean(np.abs(clean_data)):.3f}")
print(f"Clean data - Max amplitude: {np.max(np.abs(clean_data)):.3f}")

analysis.analyze_scenario_data(radar_cube, clean_data, scenario_info)

Loading scenario data...

Scenario 1 loaded:
Shape: (256, 128, 60)
Number of frames: 60
[{'scenario_id': 1, 'num_frames': 60, 'shape': (256, 128, 60)}]
Input shapes - radar_cube: (256, 128, 60), clean_data: (256, 128, 60)
Data types - radar_cube: complex128, clean_data: complex128

Data statistics:
Radar cube - Mean amplitude: 0.132
Radar cube - Max amplitude: 0.247
Clean data - Mean amplitude: 0.129
Clean data - Max amplitude: 0.198

Data Analysis Summary:
Total number of scenarios: 1
Total number of frames: 60
Data dimensions: (256, 128, 60)

Scenario 1:
Frames: 60
Input SIR: 12.91 dB


In [ ]:
# Analyze original data
print("Analyzing original data...")
analysis.analyze_rd_processing(radar_cube, clean_data)

In [ ]:
# Data preparation
full_dataset = RadarDataset(radar_cube, clean_data)
val_size = int(len(full_dataset) * 0.2)
print(f"size of validation: {val_size}")
train_size = len(full_dataset) - val_size
train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])

batch_size = 4
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

# Training preparation
""""""""" For dual-path autoencoder usage """""""""
# Initialize model
print("Initializing model......")
num_samples, num_chirps = radar_cube.shape[0:2]
# model = dualpath_autoencoder.DualPathRadarAutoencoder(num_samples=num_samples, num_chirps=num_chirps)
model = ref_dual_path_autoencoder.RefDualPathAutoencoder()
""""""""" For dual-path autoencoder usage """""""""

# # Create dummy inputs for both paths
# batch_size = 4
# summary_size = [
#     (batch_size, 2, 256, 128),  # Raw signal input: [batch, channels, samples, chirps]
#     (batch_size, 2, 256, 128)   # RD map input: [batch, channels, samples, chirps]
# ]
# print(f"Summary input shapes: {summary_size}")
# summary(model, summary_size)

complex_to_real() triggered:
Real part range: [-0.246, 0.246]
Imag part range: [-0.246, 0.246]
complex_to_real() triggered:
Real part range: [-0.198, 0.198]
Imag part range: [-0.198, 0.198]
Dataset shapes after processing:
Interference data shape: (2, 256, 128, 60)
Clean data shape: (2, 256, 128, 60)
Number of frames: 60
size of validation: 12
Initializing model......


' For dual-path autoencoder usage '

In [ ]:
print("\nStarting model training...")
trained_model, history, memory_usage, training_time = train_refdual_model(
    model, 
    train_loader, 
    val_loader,
    num_epochs=50,
    learning_rate=0.0005
)

visualization.plot_training_history(history)

In [ ]:
importlib.reload(evaluation)
# Evaluate model, get training results
print("\nEvaluating model performance...")
for idx, info in enumerate(scenario_info):
    start_frame = sum(s['num_frames'] for s in scenario_info[:idx])
    scenario_radar = radar_cube[:, :, start_frame:start_frame + info['num_frames']]
    scenario_clean = clean_data[:, :, start_frame:start_frame + info['num_frames']]

    print(f"\nEvaluating Scenario {info['scenario_id']}:")
    # train_results = evaluate_reconstruction(trained_model, scenario_radar, scenario_clean)

    """"""""" For dual-path autoencoder usage """""""""
    # After training
    print("\nEvaluating model performance...")
    train_results, reconstructed_data = evaluation_ref_dp.evaluate_refdual_reconstruction(
    trained_model, 
    scenario_radar, 
    scenario_clean,
    frame_idx=0
)
    """"""""" For dual-path autoencoder usage """""""""

    # reconstructed_cube, train_results = evaluate_and_save_results(
    #     trained_model,
    #     scenario_radar,
    #     scenario_clean
    # )

# Store original metrics
train_ref_metrics_2t1 = train_results
# print(train_metrics)

In [ ]:

model = ref_dual_path_autoencoder.RefDualPathAutoencoder()


model.count_parameters()


time_input = torch.randn(8, 2, 256, 128)  
rd_input = torch.randn(8, 2, 256, 128)    


output = model(time_input, rd_input)


print(f"output shape: {output.shape}")